__This notebook will be mainly used for the capstone project.__

In [1]:
import pandas as pd
import numpy as np

In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


### Week 3  Assignment
#### Segmenting and Clustering Neighborhoods in the city of Toronto, Canada

import some necessary libraries

In [3]:
import string
from bs4 import BeautifulSoup
import requests
import geocoder
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
from sklearn.cluster import KMeans
import csv

set display option

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## 1.web scraping

create the file in order to store the data scrapied from wiki

In [5]:
csv_file = open('pccan.csv','w')
csv_writer = csv.writer(csv_file)

In [6]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

In [7]:
soup = BeautifulSoup(source,'lxml')

Let's browse the code of the html

In [8]:
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":920980179,"wgRevisionId":920980179,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":!1,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNames

In [9]:
table = soup.find('table')

In [10]:
table.find_all('tr')

[<tr>
 <th>Postcode</th>
 <th>Borough</th>
 <th>Neighbourhood
 </th></tr>, <tr>
 <td>M1A</td>
 <td>Not assigned</td>
 <td>Not assigned
 </td></tr>, <tr>
 <td>M2A</td>
 <td>Not assigned</td>
 <td>Not assigned
 </td></tr>, <tr>
 <td>M3A</td>
 <td><a href="/wiki/North_York" title="North York">North York</a></td>
 <td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
 </td></tr>, <tr>
 <td>M4A</td>
 <td><a href="/wiki/North_York" title="North York">North York</a></td>
 <td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
 </td></tr>, <tr>
 <td>M5A</td>
 <td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
 <td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
 </td></tr>, <tr>
 <td>M5A</td>
 <td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
 <td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
 </td></tr>, <tr>
 <td>M6A</td>
 <td

In [11]:
for line in table.find_all('tr'):
    Postcode = line.text.split('\n')[1]
    print(Postcode)
    Borough = line.text.split('\n')[2]
    print(Borough)
    Neighbourhood = line.text.split('\n')[3]
    print(Neighbourhood)
    csv_writer.writerow([Postcode,Borough,Neighbourhood])
    #print()

Postcode
Borough
Neighbourhood
M1A
Not assigned
Not assigned
M2A
Not assigned
Not assigned
M3A
North York
Parkwoods
M4A
North York
Victoria Village
M5A
Downtown Toronto
Harbourfront
M5A
Downtown Toronto
Regent Park
M6A
North York
Lawrence Heights
M6A
North York
Lawrence Manor
M7A
Queen's Park
Not assigned
M8A
Not assigned
Not assigned
M9A
Etobicoke
Islington Avenue
M1B
Scarborough
Rouge
M1B
Scarborough
Malvern
M2B
Not assigned
Not assigned
M3B
North York
Don Mills North
M4B
East York
Woodbine Gardens
M4B
East York
Parkview Hill
M5B
Downtown Toronto
Ryerson
M5B
Downtown Toronto
Garden District
M6B
North York
Glencairn
M7B
Not assigned
Not assigned
M8B
Not assigned
Not assigned
M9B
Etobicoke
Cloverdale
M9B
Etobicoke
Islington
M9B
Etobicoke
Martin Grove
M9B
Etobicoke
Princess Gardens
M9B
Etobicoke
West Deane Park
M1C
Scarborough
Highland Creek
M1C
Scarborough
Rouge Hill
M1C
Scarborough
Port Union
M2C
Not assigned
Not assigned
M3C
North York
Flemingdon Park
M3C
North York
Don Mills South
M

In [12]:
csv_file.close()

## 2. clean the raw data to create a dataframe as requested

In [13]:
raw_data = pd.read_csv('pccan.csv')

In [14]:
raw_data = raw_data[~(raw_data['Borough']=='Not assigned')].reset_index(drop= True)

In [15]:
pd_data = raw_data.groupby(['Postcode','Borough']).sum().reset_index().set_index('Postcode')

In [16]:
pd_data['Neighbourhood'] = raw_data.groupby('Postcode')['Neighbourhood'].agg(lambda x:','.join(x))

In [18]:
embed_data = pd_data.reset_index()

In [21]:
embed_data.rename(columns={'Postcode':'PostalCode'},inplace = True)

Let's check the dataframe

In [23]:
embed_data.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


print the number of rows of the dataframe

In [24]:
print('There are {} rows in the dataframe'.format(embed_data.shape[0]))

There are 103 rows in the dataframe


## 3. get the latitude and longitude coordinates of each neighborhood  
* Due to the unreliability of geocoder, I used the csv file that has the geographical coordinates of each postal code

In [25]:
coor = pd.read_csv('Geospatial_Coordinates.csv').set_index('Postal Code')

In [27]:
coor.rename(index={'Postal Code':'PostalCode'},inplace = True)

In [30]:
embed_data = embed_data.set_index('PostalCode')

merge the coordinate information file with the embed_data dataframe

In [31]:
toronto_data = pd.merge(embed_data,coor,left_index=True,right_index=True).reset_index()

In [32]:
toronto_data.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [33]:
#geographical coordinates of Toronto
latitude = 43.653226
longitude = -79.383184

In [34]:
# create map of Torontousing latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

## 4. explore neighborhoods in Toronto using Foursquare

In [35]:
CLIENT_ID = 'AAOIDTJMJGHIQSS4CK51C2HZD52KB5U45JLQX0YIYW0BDPF3' # your Foursquare ID
CLIENT_SECRET = 'X5TCGWCVL3FSRJSUMYMOU3PJN0JZAZAAAVD4LKAV33S1VQWC' # your Foursquare Secret
VERSION = '20180605'
radius = 500
LIMIT = 100

In [36]:
def getNearbyVenues(names,latitudes,longitudes,radius = 500):
    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT) 
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        #return only relevant information for each nearby venue
        venues_list.append([(name,
                           lat,
                           lng,
                           v['venue']['name'],
                           v['venue']['location']['lat'],
                           v['venue']['location']['lng'],
                           v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    # item for (venue_list in venues_lists) for item in venue_list
    nearby_venues.columns = ['Neighbourhood',
                             'Neighbourhood Latitude',
                             'Neighbourhood Longitude',
                             'Venue',
                             'Venue Latitude',
                             'Venue Longitude',
                             'Venue Category']
    return(nearby_venues)

In [38]:
toronto_venues = getNearbyVenues(toronto_data['Neighbourhood'],
                                  toronto_data['Latitude'],
                                  toronto_data['Longitude'])

Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beaches West,Indi

In [42]:
toronto_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,"Guildwood,Morningside,West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


check how many venues were returned for each neighborhood

In [44]:
toronto_venues.groupby('Neighbourhood').count().sort_values(by='Venue')[:20]

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Rouge,Malvern",1,1,1,1,1,1
"Cloverdale,Islington,Martin Grove,Princess Gardens,West Deane Park",1,1,1,1,1,1
Weston,1,1,1,1,1,1
"Emery,Humberlea",1,1,1,1,1,1
"Silver Hills,York Mills",1,1,1,1,1,1
"Highland Creek,Rouge Hill,Port Union",1,1,1,1,1,1
Scarborough Village,1,1,1,1,1,1
"Moore Park,Summerhill East",2,2,2,2,2,2
Parkwoods,2,2,2,2,2,2


__discard the neighborhood whoes number of venues is less than 10__

In [56]:
venues_10 = toronto_venues.groupby('Neighbourhood').count()

In [58]:
venues_10 = venues_10[venues_10['Venue']<10].reset_index()

In [60]:
Nless10 = []
for i in venues_10['Neighbourhood']:
    Nless10.append(i)

In [63]:
toronto_venues = toronto_venues[~(toronto_venues['Neighbourhood'].isin(Nless10))]

In [64]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 258 uniques categories.


In [66]:
print('There are {} uniques neighborhood.'.format(len(toronto_venues['Neighbourhood'].unique())))

There are 49 uniques neighborhood.


## 2. Analyze Each Neighborhood

In [67]:
# one-hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']],prefix = "",prefix_sep="")

In [70]:
toronto_onehot['Neighbourhood']=toronto_venues['Neighbourhood']

In [71]:
# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

__group rows by neighbourhood and by taking the mean of the frequency of occurrence of each category__

In [73]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.030000,0.0,0.0,0.010000,0.01,0.0,0.03,0.0,0.0,0.000000,0.0,0.000000,0.020000,0.000000,0.04,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.01,0.0,0.01,0.020000,0.0,0.000000,0.0,0.01,0.03,0.01,0.0,0.

In [74]:
toronto_grouped.shape

(49, 259)

__write a function to sort the venues in descending order__

In [75]:
def return_most_common_venues(row,num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending = False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

__Now let's create the new dataframe and display the top 10 venues for each neighborhood.__

In [76]:
num_top_venues = 10

indicators = ['st','nd','rd']

#create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1,indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [77]:
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

In [78]:
for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

In [79]:
neighborhoods_venues_sorted.shape

(49, 11)

## 3.Cluster Neighborhoods

In [82]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 2, 2, 2, 1, 1, 2, 1, 1, 1])

__Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.__

In [83]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [89]:
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,"Adelaide,King,Richmond",Coffee Shop,Café,Steakhouse,Bar,Thai Restaurant,Burger Joint,Sushi Restaurant,Restaurant,Hotel,American Restaurant
1,2,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Pizza Place,Fried Chicken Joint,Sandwich Place,Discount Store,Liquor Store,Fast Food Restaurant,Beer Store,Japanese Restaurant,Pharmacy
2,2,"Bathurst Manor,Downsview North,Wilson Heights",Coffee Shop,Frozen Yogurt Shop,Pharmacy,Middle Eastern Restaurant,Restaurant,Deli / Bodega,Sandwich Place,Diner,Bridal Shop,Shopping Mall
3,2,"Bedford Park,Lawrence Manor East",Italian Restaurant,Coffee Shop,Fast Food Restaurant,Thai Restaurant,Juice Bar,Indian Restaurant,Liquor Store,Comfort Food Restaurant,Sushi Restaurant,Greek Restaurant
4,1,Berczy Park,Coffee Shop,Cocktail Bar,Farmers Market,Café,Cheese Shop,Bakery,Seafood Restaurant,Beer Bar,Steakhouse,Jazz Club


In [91]:
neighborhoods_venues_sorted.shape

(49, 12)

In [92]:
toronto_merged = toronto_data[~(toronto_data['Neighbourhood'].isin(Nless10))]

__merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood__

In [103]:
toronto_merged = pd.merge(neighborhoods_venues_sorted,toronto_merged,how = 'inner',left_on = 'Neighbourhood',right_on = 'Neighbourhood')

In [104]:
toronto_merged.head()

,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,PostalCode,Borough,Latitude,Longitude
0,1,"Adelaide,King,Richmond",Coffee Shop,Café,Steakhouse,Bar,Thai Restaurant,Burger Joint,Sushi Restaurant,Restaurant,Hotel,American Restaurant,M5H,Downtown Toronto,43.650571,-79.384568
1,2,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Pizza Place,Fried Chicken Joint,Sandwich Place,Discount Store,Liquor Store,Fast Food Restaurant,Beer Store,Japanese Restaurant,Pharmacy,M9V,Etobicoke,43.739416,-79.588437
2,2,"Bathurst Manor,Downsview North,Wilson Heights",Coffee Shop,Frozen Yogurt Shop,Pharmacy,Middle Eastern Restaurant,Restaurant,Deli / Bodega,Sandwich Place,Diner,Bridal Shop,Shopping Mall,M3H,North York,43.754328,-79.442259
3,2,"Bedford Park,Lawrence Manor East",Italian Restaurant,Coffee Shop,Fast Food Restaurant,Thai Restaurant,Juice Bar,Indian Restaurant,Liquor Store,Comfort Food Restaurant,Sushi Restaurant,Greek Restaurant,M5M,North York,43.733283,-79.419750
4,1,Berczy Park,Coffee Shop,Cocktail Bar,Farmers Market,Café,Cheese Shop,Bakery,Seafood Restaurant,Beer Bar,Steakhouse,Jazz Club,M5E,Downtown Toronto,43.644771,-79.373306


visualize the resulting clusters

In [106]:
toronto_merged['Cluster Labels'].unique()

array([1, 2, 0, 3], dtype=int64)

In [108]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 4. examine clcusters

In [109]:
toronto_merged.columns

Index(['Cluster Labels', 'Neighbourhood', '1st Most Common Venue',
       '2nd Most Common Venue', '3rd Most Common Venue',
       '4th Most Common Venue', '5th Most Common Venue',
       '6th Most Common Venue', '7th Most Common Venue',
       '8th Most Common Venue', '9th Most Common Venue',
       '10th Most Common Venue', 'PostalCode', 'Borough', 'Latitude',
       'Longitude'],
      dtype='object')

In [117]:
fixcol = ['Borough','Neighbourhood', '1st Most Common Venue',
       '2nd Most Common Venue', '3rd Most Common Venue',
       '4th Most Common Venue', '5th Most Common Venue',
         '6th Most Common Venue', '7th Most Common Venue',
       '8th Most Common Venue', '9th Most Common Venue',
       '10th Most Common Venue']

__one thing special of cluster 0 might be that many nearby venues are entertainment ones like park and music venues__

In [118]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, fixcol]

,Borough,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Downtown Toronto,Christie,Grocery Store,Café,Park,Italian Restaurant,Coffee Shop,Baby Store,Convenience Store,Restaurant,Diner,Nightclub
27,West Toronto,"High Park,The Junction South",Café,Mexican Restaurant,Thai Restaurant,Music Venue,Cajun / Creole Restaurant,Bar,Bakery,Fried Chicken Joint,Diner,Furniture / Home Store


__neighbourhoods in cluster 1 are almost surrounded by coffee shop, and most of the nearby venues are about eating__

In [119]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, fixcol]

,Borough,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,"Adelaide,King,Richmond",Coffee Shop,Café,Steakhouse,Bar,Thai Restaurant,Burger Joint,Sushi Restaurant,Restaurant,Hotel,American Restaurant
4,Downtown Toronto,Berczy Park,Coffee Shop,Cocktail Bar,Farmers Market,Café,Cheese Shop,Bakery,Seafood Restaurant,Beer Bar,Steakhouse,Jazz Club
5,West Toronto,"Brockton,Exhibition Place,Parkdale Village",Café,Yoga Studio,Performing Arts Venue,Breakfast Spot,Coffee Shop,Nightclub,Convenience Store,Italian Restaurant,Pet Store,Gym
7,Downtown Toronto,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Airport Lounge,Airport Terminal,Plane,Bar,Harbor / Marina,Coffee Shop,Sculpture Garden,Boutique,Boat or Ferry
8,Downtown Toronto,"Cabbagetown,St. James Town",Coffee Shop,Chinese Restaurant,Restaurant,Grocery Store,Market,Café,Pizza Place,Bakery,Pub,Italian Restaurant
9,Mississauga,Canada Post Gateway Processing Centre,Coffee Shop,Hotel,American Restaurant,Middle Eastern Restaurant,Fried Chicken Joint,Sandwich Place,Mediterranean Restaurant,Burrito Place,Discount Store,Dog Run
10,Downtown Toronto,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Middle Eastern Restaurant,Burger Joint,Ice Cream Shop,Sandwich Place,Bar,Juice Bar,Salad Place
11,Downtown Toronto,"Chinatown,Grange Park,Kensington Market",Bar,Chinese Restaurant,Vegetarian / Vegan Restaurant,Café,Bakery,Mexican Restaurant,Vietnamese Restaurant,Dumpling Restaurant,Coffee Shop,Burger Joint
13,Downtown Toronto,Church and Wellesley,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Yoga Studio,Café,Pub,Hotel,Burger Joint
16,Downtown Toronto,"Commerce Court,Victoria Hotel",Coffee Shop,Café,Hotel,American Restaurant,Restaurant,Gastropub,Deli / Bodega,Italian Restaurant,Steakhouse,Gym


__the appearance of fastfood restaurants is really high in neighbourhoods in cluster 2__

In [120]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, fixcol]

,Borough,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Pizza Place,Fried Chicken Joint,Sandwich Place,Discount Store,Liquor Store,Fast Food Restaurant,Beer Store,Japanese Restaurant,Pharmacy
2,North York,"Bathurst Manor,Downsview North,Wilson Heights",Coffee Shop,Frozen Yogurt Shop,Pharmacy,Middle Eastern Restaurant,Restaurant,Deli / Bodega,Sandwich Place,Diner,Bridal Shop,Shopping Mall
3,North York,"Bedford Park,Lawrence Manor East",Italian Restaurant,Coffee Shop,Fast Food Restaurant,Thai Restaurant,Juice Bar,Indian Restaurant,Liquor Store,Comfort Food Restaurant,Sushi Restaurant,Greek Restaurant
6,East Toronto,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Comic Shop,Park,Recording Studio
14,Scarborough,"Clairlea,Golden Mile,Oakridge",Bus Line,Bakery,Fast Food Restaurant,Intersection,Metro Station,Bus Station,Soccer Field,Park,Costume Shop,Concert Hall
15,Scarborough,"Clarks Corners,Sullivan,Tam O'Shanter",Pizza Place,Thai Restaurant,Noodle House,Chinese Restaurant,Shopping Mall,Fast Food Restaurant,Bank,Italian Restaurant,Fried Chicken Joint,Pharmacy
17,Central Toronto,Davisville,Dessert Shop,Sandwich Place,Pizza Place,Coffee Shop,Gym,Italian Restaurant,Café,Sushi Restaurant,Seafood Restaurant,Restaurant
20,West Toronto,"Dovercourt Village,Dufferin",Pharmacy,Supermarket,Bakery,Pizza Place,Art Gallery,Middle Eastern Restaurant,Music Venue,Park,Café,Brewery
28,Etobicoke,"Humber Bay Shores,Mimico South,New Toronto",Pizza Place,Bakery,Fried Chicken Joint,Flower Shop,Fast Food Restaurant,Liquor Store,Mexican Restaurant,Pharmacy,Restaurant,Café
29,Etobicoke,"Kingsway Park South West,Mimico NW,The Queensw...",Hardware Store,Convenience Store,Supplement Shop,Burrito Place,Burger Joint,Sandwich Place,Discount Store,Thrift / Vintage Store,Flower Shop,Bakery


__Cluster 3 seems to be a venue for shopping__

In [121]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, fixcol]

,Borough,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
31,North York,"Lawrence Heights,Lawrence Manor",Clothing Store,Furniture / Home Store,Accessories Store,Sporting Goods Shop,Women's Store,Gift Shop,Event Space,Miscellaneous Shop,Shoe Store,Boutique


### conclusion  
1. __one thing special of cluster 0 might be that many nearby venues are entertainment ones like park and music venues__  
2. __neighbourhoods in cluster 1 are almost surrounded by coffee shop, and most of the nearby venues are about eating__  
3. __the appearance of fastfood restaurants is really high in neighbourhoods in cluster 2__  
4. __Cluster 3 seems to be venues for shopping__